In [1]:
import tensorflow as tf

try:
    interpreter = tf.lite.Interpreter(model_path="ted_tflite_models/ted_model.tflite")
    interpreter.allocate_tensors()
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")

2024-08-24 22:23:46.011903: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-24 22:23:46.029495: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-24 22:23:46.029527: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-24 22:23:46.040101: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-24 22:23:47.179200: W tensorflow/compiler/tf

Model loaded successfully.


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [2]:
# Получаем информацию о всех тензорах
tensor_details = interpreter.get_tensor_details()

# Поиск и вывод информации о tensor#58
tensor_index = 53  # Это индекс тензора, указанный в предупреждении

for tensor in tensor_details:
    if tensor['index'] == tensor_index:
        print(f"Tensor found: {tensor}")
        break
else:
    print(f"Tensor with index {tensor_index} not found.")

Tensor found: {'name': 'ted/up_conv_block_2/sequential_2/conv2d_transpose_2/BiasAdd;ted/up_conv_block_2/sequential_2/conv2d_transpose_2/conv2d_transpose;ted/up_conv_block_2/sequential_2/conv2d_transpose_2/BiasAdd/ReadVariableOp', 'index': 53, 'shape': array([  1, 176, 176,  16], dtype=int32), 'shape_signature': array([  1, 176, 176,  16], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [3]:
import pathlib

model = tf.keras.layers.TFSMLayer('best_model', call_endpoint='serving_default')

fixed_input = tf.keras.Input(shape=(352, 352, 3), batch_size=1)

fixed_model = tf.keras.Model(inputs=fixed_input, outputs=model(fixed_input))

converter = tf.lite.TFLiteConverter.from_keras_model(fixed_model)
tflite_model = converter.convert()

tflite_models_dir = pathlib.Path("ted_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

tflite_model_file = tflite_models_dir/"ted_model.tflite"
tflite_model_file.write_bytes(tflite_model)


2024-08-24 22:24:02.712953: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-24 22:24:03.042975: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-08-24 22:24:03.043429: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

INFO:tensorflow:Assets written to: /tmp/tmpstciv092/assets


INFO:tensorflow:Assets written to: /tmp/tmpstciv092/assets
W0000 00:00:1724527446.749908   12233 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1724527446.749965   12233 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-08-24 22:24:06.751286: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpstciv092
2024-08-24 22:24:06.771984: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-08-24 22:24:06.772021: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpstciv092
2024-08-24 22:24:06.833077: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-08-24 22:24:06.839673: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-08-24 22:24:07.091293: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: /tmp/tmpstciv092
2024-0

246164

In [8]:
tf.lite.experimental.Analyzer.analyze(model_content=tflite_model)


=== TFLite ModelAnalyzer ===

Your TFLite model has '1' subgraph(s). In the subgraph description below,
T# represents the Tensor numbers. For example, in Subgraph#0, the CONV_2D op takes
tensor #0 and tensor #37 and tensor #38 as input and produces tensor #39 as output.

Subgraph#0 main(T#0) -> [T#46, T#65, T#57, T#55]
  Op#0 CONV_2D(T#0, T#37, T#38) -> [T#39]
  Op#1 CONV_2D(T#39, T#33, T#8) -> [T#40]
  Op#2 CONV_2D(T#40, T#32, T#7) -> [T#41]
  Op#3 CONV_2D(T#41, T#31, T#17) -> [T#42]
  Op#4 MAX_POOL_2D(T#42) -> [T#43]
  Op#5 CONV_2D(T#43, T#30, T#16) -> [T#44]
  Op#6 CONV_2D(T#42, T#29, T#6) -> [T#45]
  Op#7 TRANSPOSE_CONV(T#35[1, 352, 352, 1], T#21, T#45, T#15) -> [T#46]
  Op#8 CONV_2D(T#40, T#28, T#14) -> [T#47]
  Op#9 ADD(T#43, T#47) -> [T#48]
  Op#10 CONV_2D(T#48, T#27, T#5) -> [T#49]
  Op#11 CONV_2D(T#49, T#26, T#16) -> [T#50]
  Op#12 ADD(T#50, T#44) -> [T#51]
  Op#13 CONV_2D(T#51, T#1, T#4) -> [T#52]
  Op#14 TRANSPOSE_CONV(T#34[1, 176, 176, 16], T#20, T#52, T#13) -> [T#53]
  Op#